In [ ]:
# %load sweeper.py
import torch
from torch import nn, optim
import torchvision.models as models
from hyperopt import fmin, tpe, hp
from sklearn.metrics import accuracy_score
from preprocess import get_loader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define the search space for hyperparameters
space = {
    #'lr': hp.loguniform('lr', -4, -2),
    'weight_decay': hp.loguniform('weight_decay', -6, -2),
    #'momentum': hp.uniform('momentum', 0.8, 0.99),
}

def objective(params):

  weights = models.ResNet50_Weights.DEFAULT
  model = models.resnet50(weights=weights)

  for param in model.parameters():
    param.requires_grad = False

  n_features = model.fc.in_features
  n_cls = 10

  model.fc = nn.Linear(n_features, n_cls)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.04297047718491614, weight_decay=params['weight_decay'])

  trainloader, validloader, _ = get_loader(batch_size=128)

  model.to(device)

  # Train the model for one epoch

  model.train()
  for x, y in trainloader:
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    out = model(x)
    loss = loss_fn(out, y)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.inference_mode():
      y_true = []
      y_pred = []

      for x, y in validloader:
        x, y = x.to(device), y.to(device)
        out = model(x)
        _, pred = torch.max(out, 1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())
    accuracy = accuracy_score(y_true, y_pred)
    return -accuracy

# run the optimzation algorithm
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=300)
print(best)


In [1]:
# %load train.py
#%%
import torch
from torch import nn, optim
import torchvision.models as models
from preprocess import get_loader
from engine import Engine

#%%
device = 'cuda' if torch.cuda.is_available() else 'cpu'

weights = models.ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)

for param in model.parameters():
  param.requires_grad = False

n_features = model.fc.in_features
n_cls = 10

model.fc = nn.Linear(n_features, n_cls)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
trainloader, validloader, testloader = get_loader(batch_size=128)

#%%
engine = Engine(model, loss_fn, optimizer, device)
engine.set_loader(trainloader, validloader)
engine.train(100)


100%|██████████| 100/100 [21:42<00:00, 13.02s/it]


In [3]:
engine.train_loss, engine.val_loss # (0.13203317515159907, 0.36113059520721436) # vanilla
                                   # (0.13233724863905655, 0.3645693063735962) # vanilla
                                   # (0.016809941857661072, 0.284824013710022) # vanilla 100 epochs
                                   # (0.007854528902833792, 0.6573954820632935 # batch_size=128 lr=0.04297047718491614
                                   # (0.014901145876501687, 1.5822378396987915) # batch_size=256, lr=0.11736439305239602
                                   # (0.5129879901283666, 0.6524715423583984) # batch_size=128, lr=0.09570042181074462, weight_decay=0.012398066549914627
                                   # (0.48124869559940536, 0.39477479457855225) # batch_size=128, lr= lr=0.04297047718491614, weight_decay=0.028519667471770878

(0.016809941857661072, 0.284824013710022)

In [ ]:
import torch
'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
from sklearn.metrics import accuracy_score

trainloader, validloader, test_loader = get_loader(batch_size=128)

engine.model.eval()
with torch.inference_mode():
  y_true = []
  y_pred = []

  for x, y in test_loader:
    x, y = x.to(device), y.to(device)
    out = engine.model(x)
    _, pred = torch.max(out, 1)
    y_true.extend(y.cpu().numpy())
    y_pred.extend(pred.cpu().numpy())
  accuracy = accuracy_score(y_true, y_pred)
  print(accuracy)

0.94


In [9]:
torch.save(engine.model.state_dict(), 'model/model_params.pth')